## လိုအပ်သောစာကြည့်တိုက်များကို တင်သွင်းပါ  
Phi-4 မော်ဒယ်ကို တင်ပြီး အသုံးပြုရန် လိုအပ်သော PyTorch နှင့် transformers စာကြည့်တိုက်များကို တင်သွင်းပါ။  


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## အလွတ်စတင်အမှတ်စဉ် သတ်မှတ်ခြင်း  
အလွတ်စတင်အမှတ်စဉ်ကို သတ်မှတ်ပါ၊ ဒါမှမဟုတ် အမျိုးမျိုးသော အကြိမ်ကြိမ်တွင် တူညီသော ရလဒ်များရရှိစေရန် အာမခံနိုင်ပါမည်။


In [ ]:
torch.random.manual_seed(0)

## Phi-4-mini-flash-reasoning မော်ဒယ်နှင့် Tokenizer ကို Load လုပ်ခြင်း  
Microsoft ရဲ့ Phi-4-mini-flash-reasoning မော်ဒယ်နှင့် အဆင်ပြေတဲ့ tokenizer ကို Hugging Face မှာ Load လုပ်ပါ။ မော်ဒယ်ကို CUDA ပေါ်မှာ Load လုပ်ပြီး အမြန်ဆုံး အနိမ့်ဆုံး အကြံပေးမှုများကို ရယူနိုင်ပါမည်။


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## အဝင်မက်ဆေ့ဂျ်ကို ပြင်ဆင်ပါ  
မော်ဒယ်အတွက် စကားဝိုင်းပုံစံကို အသုံးပြု၍ ကွာဒရက်တစ်ဆိုင်ရာ သင်္ချာပြဿနာပါဝင်သော စကားဝိုင်းမက်ဆေ့ဂျ်တစ်ခု ဖန်တီးပါ။


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## တုံ့ပြန်မှု ဖန်တီးရန်  
အထွေထွေ randomness ကို ထိန်းချုပ်နိုင်ရန် temperature နှင့် top_p ကဲ့သို့သော သတ်မှတ်ထားသော parameters များကို အသုံးပြု၍ မော်ဒယ်မှ တုံ့ပြန်မှုကို ဖန်တီးပါ။


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## အထွေထွေ Output ကို Text အဖြစ် ပြန်ဖော်ခြင်း  
မူလ input token များကို မပါဝင်စေဘဲ၊ မော်ဒယ်၏ တုံ့ပြန်မှုကိုသာ လူနားလည်နိုင်သော စာသားအဖြစ် ပြန်လည်ဖော်ပြပါ။


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**ဝက်ဘ်ဆိုက်မှတ်ချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ဆိုမှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို ကျေးဇူးပြု၍ သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတည်သော ရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူပညာရှင်များမှ ဘာသာပြန်ဆိုမှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားများ သို့မဟုတ် အဓိပ္ပါယ်မှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
